In [2]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, Flatten , Dense, Dropout, Lambda
from keras.callbacks import ModelCheckpoint, CSVLogger

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
def create_model(input_shape=(66, 200, 3)):
    
    input_layer = Input(shape=input_shape)
    x = Lambda(lambda z: z/127.5 - 1.)(input_layer)
    x = Conv2D(filters=24, kernel_size=(5,5), strides=(2,2), activation='elu')(input_layer)
    x = Conv2D(filters=36, kernel_size=(5,5), strides=(2,2), activation='elu')(x)
    x = Conv2D(filters=48, kernel_size=(5,5), strides=(2,2), activation='elu')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='elu')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='elu')(x)
    
    x = Flatten()(x)
    
    x = Dense(units=100, activation='elu')(x)
    x = Dense(units=50, activation='elu')(x)
    x = Dense(units=10, activation='elu')(x)
    output_layer = Dense(units=1)(x)
    
    model = Model(input=input_layer, output=output_layer)
    
    print(model.summary())
    

In [8]:
create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
__________

C:\Users\windo\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
